In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score

# Load your custom dataset from a CSV file
csv_path = 'datasets/mizo-news-14k.csv'
df = pd.read_csv(csv_path)

# Split the dataset into training and testing sets with stratification
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Load pre-trained DistilBERT model and tokenizer
model_name = "robzchhangte/MizBERT"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Adjust num_labels based on your categories

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx]['text'])
        label = int(self.dataframe.iloc[idx]['label'])

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# Create instances of the custom dataset
train_dataset = CustomDataset(train_df, tokenizer, max_length=128)
test_dataset = CustomDataset(test_df, tokenizer, max_length=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./transformers_sequence_classification",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Define Trainer with custom compute_metrics function
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = torch.argmax(torch.from_numpy(predictions), axis=1).numpy()
    
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')

    return {"accuracy": acc, "f1_score": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,No log,0.186052,0.946200,0.946093
2,0.282200,0.176616,0.955279,0.955478
3,0.114800,0.177837,0.955279,0.955318


/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: Us

{'eval_loss': 0.1778368502855301, 'eval_accuracy': 0.9552790854068595, 'eval_f1_score': 0.9553176656529324, 'eval_runtime': 24.3469, 'eval_samples_per_second': 122.151, 'eval_steps_per_second': 3.82, 'epoch': 3.0}
